In [127]:
import math
import random
import time

In [128]:
class Problem:
    """Esta es una clase abstracta para un problema.
    No se supone que debamos crear objetos de esta
    clase directamente. La idea es que otras clases
    hereden de esta para crear clases de problemas
    más especeificos."""
    
    def __init__(self, initial):
        """Esta definición es un método, es especial
        porque comienza y termina con doble piso.
        En Python, el método __init__ es utilizado
        para construir objetos de la clase.
        En este y el resto de los métodos, el primer
        parámetro es self, y nos permite referirnos
        al objeto sobre el cuál el método es invocado."""
        self.initial = initial
    
    def actions(self, state):
        """Un problema contiene un estado inicial y
        quizá una meta en particular. El método actions
        debe regresar las acciones que se pueden tomar
        en el estado dado. El resultado usualmente es
        una lista, pero si debes regresar muchas acciones,
        considera usar yield para generarlas una a la vez."""
        raise NotImplementedError
    
    def result(self, state, action):
        """El método anterior, al igual que este, tiene una
        implementación trivial. Simplemente señalamos un
        error que indica que no hay una implementación
        para el método. En una implementación de este método,
        debemos regresar el estado al que llegamos después
        de ejecutar la acción dada en el estado dado.
        La acción debe ser una calculada por
        self.actions(state)."""
        
        raise NotImplementedError
    

    def path_cost(self, c, state1, action, state2):
        """El método anterior si tenía una implementación útil,
        sigue siendo general, pero podemos pensar que en muchos
        problemas, si un estado es igual a la meta, entonces hemos
        llegado a la meta. El método path_cost también incluye una
        implementación: Se considera que llegar al estado 1 tiene
        un costo c. Regresa el costo de un camino de solución que
        llega al estado 2 desde estado 1 por medio de la acción dada."""
        return c + 1
    
    def random_solution(self):
        raise NotImplementedError
    
    def neighbors_of(self,state):
        raise NotImplementedError
    
    def random_neighbor(self,state):
        return self.random_choice(state)
    
    def crossover(self,state):
        raise NotImplementedError
    
    def domain_size(self):
        raise NotImplementedError
    
    #def domain(self):
    #    raise NotImplementedError
    
    def value(self, state):
        """En distintos problemas, cada estado tiene un valor asociado.
        Este método sirve para obtener este valor."""
        raise NotImplementedError
        

In [129]:
class Viaje(Problem):
    def __init__(self,people, schedule, airports):
        self.schedule = schedule
        self.airports = airports
        self.people = people
        self.destination = 'LGA'
        domain = []
        family = len(people)
        for i in range(family):
            origin = self.people[i][1]
            num_arr_flights = len(self.schedule[(origin,self.destination)])-1
            num_dep_flights = len(self.schedule[(self.destination,origin)])-1
            domain = domain + [(0,num_arr_flights)] + \
                              [(0,num_dep_flights)]
        self.domain  = domain
        

    def get_minutes(self, t):
        x = time.strptime(t, '%H:%M')
        h = x.tm_hour
        m = x.tm_min
        return 60 * h + m
    
    def value(self,s):
        # contamos el precio total de cada vuelo (ida y regreso)
        total_price = 0
        
        # nos interesa conocer el tiempo de llegada a NY mas tarde
        # y el tiempo de salida de NY mas temprano.
        latest_arrival = 0
        earliest_departure = 24 * 60
        earliest_arrival = 24 * 60
        latest_departure = 0
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.schedule[(origin, self.destination)][s[2*i]]
            ret_flight = self.schedule[(self.destination, origin)][s[2*i+1]]
            
            total_price += out_flight[2] # vuelo de ida
            total_price += ret_flight[2] # vuelo de regreso

            out_arrival = self.get_minutes(out_flight[1])
            ret_departure = self.get_minutes(ret_flight[0])
            
            earliest_arrival = min(earliest_arrival, out_arrival)
            latest_arrival = max(latest_arrival, out_arrival)
            earliest_departure = min(earliest_departure, ret_departure)
            latest_departure = max(latest_departure, ret_departure)
        
        if latest_arrival > earliest_departure + 60:
            family_time = (math.e ** (earliest_departure - latest_arrival - 60))+ 1000
        else:
            family_time = latest_arrival - earliest_departure

        out_waiting_time = latest_arrival - earliest_arrival
        ret_waiting_time = latest_departure - earliest_departure
        family_time = earliest_departure - latest_arrival
        
        return (
            total_price + 
            out_waiting_time + 
            ret_waiting_time -
            family_time
        )
    
    def random_solution(self):
        return [random.randint(r[0], r[1]) for r in self.domain]

    
    def neighbors_of(self, state):
        neighbors = []
        for i in range(len(self.domain)):
            if state[i] > self.domain[i][0]:
                neighbors.append(state[0:i] + [state[i] - 1] + state[i+1:])
            if state[i] < self.domain[i][1]:
                neighbors.append(state[0:i] + [state[i] + 1] + state[i+1:])
        return neighbors
    
    def mutate(self, s):
        return random.choice(self.neighbors_of(s))
    
    def crossover(self, s1, s2):
        i = random.randint(1, len(s1)-2)
        return s1[:i] + s2[i:]
    
    def random_neighbor(self,s):
        all_flights = [i for i in range(2*len(self.people))]
        one_neighbor = [i for i in all_flights if s[i] == 0 or  (i % 2 == 0 and s[i] == len(self.schedule \
                            [(self.people[ i//2][1],self.destination)])-1) or (i % 2 == 1 and s[i] == len(self.schedule \
                            [(self.destination,self.people[ i//2][1])])-1)]
        two_neighbors = [item for item in all_flights if item not in one_neighbor]
        
        num_neighbors = 2*len(two_neighbors) + len(one_neighbor)

        x = random.random()
        if x < 2*len(two_neighbors) / num_neighbors:
            y= random.choice(two_neighbors)
            s[y] = random.choice([s[y]+1,s[y]-1])

        else:
            y = random.choice(one_neighbor)
            if s[y] == 0:
                s[y] = 1
            elif y % 2 == 0:
                s[y] = len(self.schedule[(self.people[ y//2][1],self.destination)])-2
            else:
                s[y] = len(self.schedule[(self.destination,self.people[ y//2][1])])-2
        return s

    def domain_size(self):
        size = 1
        for i in range(len(self.people)):
            size = len(self.schedule[(self.people[ i//2][1],self.destination)]) * \
                   len(self.schedule[(self.people[i//2][1],self.destination)]) * size
        return size

In [130]:
class OptimazationMethod:
    def solve(slef, Problem):
        raise NotImplementedError
    


In [131]:
class ExhaustiveSearch(OptimazationMethod):
    def solve(self,Problem):
        if Problem.domain_size:
            return "Impossible to calculate with this optimization method in this computer."
        else:
            solution = None
            min_cost = float('inf')
            for element in Problem.domain():
                if min(element) < min_cost:
                    solution = element
                    min_cost = Problem.value(solution)
        return solution


In [132]:
class SolveRandomly(OptimazationMethod):
    def __init__(self, repeats: int = 1000):
        self.repeats = repeats
        
    def solve(self,Problem,):
        best_cost = float('inf')
        best_sol = None
    
        for _ in range(self.repeats):
            s = Problem.random_solution()
            c = Problem.value(s)
            if c <= best_cost:
                best_cost = c
                best_sol = s
    
        return best_sol

In [133]:
class HillClimbing(OptimazationMethod):
    def solve(self, Problem):
        solution = Problem.random_solution()
        
        while True:
            neighbors = Problem.neighbors_of(solution)
            if len(neighbors) == 0:
                return solution
            cost = Problem.value(solution)
            best_neighbor = min(neighbors, key=Problem.value)
            neighbor_cost = Problem.value(best_neighbor)
            
            if cost < neighbor_cost:  
                return solution
            
            solution = best_neighbor

In [134]:
class Annealing(OptimazationMethod):
    def __init__(self,Ti=10000,Tf=.1,alpha=.95):
        self.Ti = Ti
        self.Tf = Tf
        self.alpha = alpha

    def solve(self,Problem):
        solution = Problem.random_solution()
        cost = Problem.value(solution)
        T=self.Ti
        while T > self.Tf:
            if Problem.neighbors_of(solution) == 0:
                return solution
            neighbor = Problem.random_neighbor(solution)
            neighbor_cost = Problem.value(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                solution = neighbor
                cost = neighbor_cost
            T = self.alpha*T
        return solution
        

In [135]:
class Evoling(OptimazationMethod):
    def __init__(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
        self.pop_size = pop_size
        self.mut_prob = mut_prob
        self.elite = elite
        self.epochs = epochs

    def solve(self, Problem):
        pop = [Problem.random_solution() for _ in range(self.pop_size)]
        top_elite = int(self.elite * self.pop_size)
        
        for epoch in range(self.epochs):
            pop.sort(key=Problem.value)
            best = pop[:top_elite]
            while len(best) < self.pop_size:
                if random.random() < self.mut_prob:
                    best.append(Problem.random_neighbor(
                        best[random.randint(0, top_elite-1)]
                    ))
                else:
                    best.append(Problem.crossover(
                        best[random.randint(0, top_elite-1)],
                        best[random.randint(0, top_elite-1)],
                    ))
            pop = best
        pop.sort(key=Problem.value)
        return pop[0]

In [136]:
people = [('Seymour', 'BOS'),
          ('Franny' , 'DAL'),
          ('Zooey'  , 'CAK'),
          ('Walt'   , 'MIA'),
          ('Buddy'  , 'ORD'),
          ('Les'    , 'OMA')]

In [137]:
AIRPORT_PATH = "./data/airport-codes.txt"
SCHEDULE_PATH = "./data/schedule.txt"

In [138]:
def load_airports(airports):
        airport = {}
        with open(airports) as f:
            f.readline()
            for line in f:
                cols = line.strip().split(',')
                iata = cols[9]
                name = cols[2]
                region = cols[6]
                municipality = cols[7]
                airport[iata] = (
                    name,
                    region,
                    municipality
                )
        return airport

In [139]:
def load_flights(path):
    flights = {}
    with open(path) as f:
        for line in f:
            origin, dest, t_depart, t_arrive, price = line.strip().split(',')
            flights.setdefault((origin, dest), [])
            flights[(origin, dest)].append((t_depart, t_arrive, int(price)))
    return flights

In [140]:
NewYork = Viaje(people, load_flights(SCHEDULE_PATH),load_airports(AIRPORT_PATH))

In [141]:
print(SolveRandomly().solve(NewYork))

[1, 4, 1, 5, 3, 7, 3, 8, 4, 8, 2, 6]


In [142]:
print(HillClimbing().solve(NewYork))

[4, 6, 3, 5, 5, 3, 3, 9, 5, 3, 6, 6]


In [143]:
print(ExhaustiveSearch().solve(NewYork))

Impossible to calculate with this optimization method in this computer.


In [167]:
print(Annealing().solve(NewYork))

[3, 9, 9, 4, 6, 4, 7, 3, 9, 1, 0, 7]


In [145]:
print(Evoling().solve(NewYork))

[3, 6, 0, 7, 1, 6, 2, 6, 4, 7, 2, 7]


In [202]:
NewYork.random_neighbor([0,2,3,4,5,0,7,8,5,0,1,2])

[0, 2, 3, 4, 5, 0, 7, 9, 5, 0, 1, 2]